## Data Preparation

In [1]:
import pandas as pd

In [2]:
movies_data_path = '../dataset/movies.csv'
finantial_data_path = '../dataset/finantials.csv'
opening_data_path = '../dataset/opening_gross.csv'

In [3]:
fin_data = pd.read_csv(finantial_data_path)
movie_data = pd.read_csv(movies_data_path)
opening_data = pd.read_csv(opening_data_path)

In [4]:
fin_data.sort_values("domestic_gross")

,Unnamed: 0,movie_name,domestic_gross,movie_link,production_budget,release_date,worldwide_gross,movie_title
3674,4331,Lies in Plain Sight,0,http://www.the-numbers.com/movie/Lies-in-Plain...,2100000,12/31/2010,0,Lies in Plain Sight
3734,4409,Grave Encounters,0,http://www.the-numbers.com/movie/Grave-Encount...,2000000,9/9/2011,2151887,Grave Encounters
4018,4752,Harrison Montgomery,0,http://www.the-numbers.com/movie/Harrison-Mont...,850000,12/31/2008,0,Harrison Montgomery
1341,1544,Bandidas,0,http://www.the-numbers.com/movie/Bandidas#tab=...,35000000,9/22/2006,19282590,Bandidas
3733,4408,Lesbian Vampire Killers,0,http://www.the-numbers.com/movie/Lesbian-Vampi...,2000000,12/29/2009,3620902,Vampire Killers
...,...,...,...,...,...,...,...,...
53,59,The Dark Knight,533345358,http://www.the-numbers.com/movie/Dark-Knight-T...,185000000,7/18/2008,1002891358,The Dark Knight
23,24,Jurassic World,652198010,http://www.the-numbers.com/movie/Jurassic-Worl...,215000000,6/12/2015,1670328025,Jurassic World
29,33,Titanic,658672302,http://www.the-numbers.com/movie/Titanic#tab=s...,200000000,12/19/1997,2207615668,Titanic
0,0,Avatar,760507625,http://www.the-numbers.com/movie/Avatar#tab=su...,425000000,12/18/2009,2783918982,Avatar


In [5]:
numeric_columns_mask = (movie_data.dtypes == float) | (movie_data.dtypes == int)
numeric_columns = [column for column in numeric_columns_mask.index if numeric_columns_mask[column]]
movie_data = movie_data[numeric_columns+['movie_title']]

In [6]:
fin_data = fin_data[['movie_title','production_budget','worldwide_gross']]

In [7]:
fin_movie_data = pd.merge(fin_data, movie_data, on= 'movie_title', how='left')

In [8]:
full_movie_data = pd.merge( opening_data,fin_movie_data, on = 'movie_title', how='left')

In [9]:
full_movie_data[(full_movie_data.worldwide_gross != 0) & (full_movie_data.worldwide_gross.notnull())].shape

(2304, 12)

In [10]:
full_movie_data = full_movie_data.drop(['movie_title','gross'],axis=1)

In [11]:
full_movie_data.columns

Index(['opening_gross', 'screens', 'production_budget', 'worldwide_gross',
       'title_year', 'aspect_ratio', 'duration', 'cast_total_facebook_likes',
       'budget', 'imdb_score'],
      dtype='object')

## Modeling

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate
import numpy as np

In [13]:
X = full_movie_data.drop(['worldwide_gross'], axis = 1)
y = full_movie_data['worldwide_gross']

In [14]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [32]:
results = cross_validate(pipeline ,X,y,return_train_score=True,cv=10)
results

{'fit_time': array([0.26471615, 0.22782278, 0.227422  , 0.2264719 , 0.22830987,
        0.2277081 , 0.22772694, 0.23181319, 0.22786999, 0.22713184]),
 'score_time': array([0.00109911, 0.00077605, 0.00081682, 0.00074911, 0.00088716,
        0.00075078, 0.00088906, 0.000916  , 0.00096798, 0.00075603]),
 'test_score': array([0.67464105, 0.85217112, 0.64424446, 0.77932919, 0.78352807,
        0.86835368, 0.75937471, 0.8768354 , 0.65135042, 0.65682418]),
 'train_score': array([0.91673951, 0.91581777, 0.9228721 , 0.91654412, 0.92172829,
        0.91476722, 0.92151444, 0.91734995, 0.92320705, 0.91766026])}

In [33]:
train_score = np.mean(results['train_score'])
test_score = np.mean(results['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9188200721676859
Test Score: 0.7546652270916326


## Hyperparameter tunning

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
param_tunning = {'core_model__n_estimators': range(20,501,20)} 

In [36]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [37]:
grid_search= GridSearchCV(estimator,
                       param_grid = param_tunning,
                       scoring='r2',
                       cv=5) 

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35,random_state= 42)

In [43]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('core_model',
                                        GradientBoostingRegressor())]),
             param_grid={'core_model__n_estimators': range(20, 501, 20)},
             scoring='r2')

In [44]:
final_result = cross_validate(grid_search.best_estimator_,X_train,y_train,return_train_score=True,cv=7)

In [45]:
train_score = np.mean(final_result['train_score'])
test_score = np.mean(final_result['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.9716090255265606
Test Score: 0.7611036745073019


In [46]:
grid_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('imputer', SimpleImputer()),
  ('core_model', GradientBoostingRegressor(n_estimators=260))],
 'verbose': False,
 'imputer': SimpleImputer(),
 'core_model': GradientBoostingRegressor(n_estimators=260),
 'imputer__add_indicator': False,
 'imputer__copy': True,
 'imputer__fill_value': None,
 'imputer__keep_empty_features': False,
 'imputer__missing_values': nan,
 'imputer__strategy': 'mean',
 'imputer__verbose': 'deprecated',
 'core_model__alpha': 0.9,
 'core_model__ccp_alpha': 0.0,
 'core_model__criterion': 'friedman_mse',
 'core_model__init': None,
 'core_model__learning_rate': 0.1,
 'core_model__loss': 'squared_error',
 'core_model__max_depth': 3,
 'core_model__max_features': None,
 'core_model__max_leaf_nodes': None,
 'core_model__min_impurity_decrease': 0.0,
 'core_model__min_samples_leaf': 1,
 'core_model__min_samples_split': 2,
 'core_model__min_weight_fraction_leaf': 0.0,
 'core_model__n_estimators': 260,
 'core_model__n_iter_no_change': None,
 'core_m

In [47]:
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor(n_estimators=220,
                                             alpha=0.9,
                                             ccp_alpha=0.0,
                                             criterion='friedman_mse',
                                             init=None,
                                             learning_rate=0.1,
                                             loss='squared_error',
                                             max_depth=3,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_iter_no_change=None,
                                             random_state=None,
                                             subsample=1.0,
                                             tol=0.0001,
                                             validation_fraction=0.1,
                                             verbose=0,
                                             warm_start=False))
])

In [48]:
estimator.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()),
                ('core_model', GradientBoostingRegressor(n_estimators=220))])

In [49]:
estimator.score(X_test, y_test)

0.7350887895469791

## Saving model

In [50]:
from joblib import dump

In [51]:
dump(estimator, '../model/model.pkl')

['../model/model.pkl']

In [52]:
X_train.columns

Index(['opening_gross', 'screens', 'production_budget', 'title_year',
       'aspect_ratio', 'duration', 'cast_total_facebook_likes', 'budget',
       'imdb_score'],
      dtype='object')